In [3]:
import pennylane as qml 
from pennylane import numpy as np 
import numpy as scinp 
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [10, 10]
import torch
from torch.nn.init import xavier_uniform_
from torch import nn 

In [4]:
latent_dimensionality=2

In [5]:
dev=qml.device("qulacs.simulator",wires=latent_dimensionality,gpu=True)

In [6]:
def state_preparation(latent_variables):
    for i in range(len(latent_variables)):
        qml.RX(latent_variables[i],wires=i)

In [7]:
def layer(W):
    num_wires=len(W)
    for i in range(num_wires):
        random_rot=np.random.choice([qml.RX,qml.RY,qml.RZ])
        random_rot(W[i],wires=i)
        
    for i in range(num_wires-1):
        qml.CNOT(wires=[i,i+1])
        

In [8]:
@qml.qnode(dev,interface='torch')
def variational_circuit(latent_variables,weights):
    # The number of wires is exactly equal to 
    # the dimensionality of the latent variables.
    
    state_preparation(latent_variables)

    for W in weights:
        layer(W)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(len(latent_variables))]

# We test our variational circuit by generating random variables and a glorot uniform distributed weights array, and drawing the circuit and evaluating it. 

In [9]:
latent_variables=scinp.random.uniform(-np.pi,np.pi,size=(latent_dimensionality,))
latent_variables

array([0.18158041, 1.69477679])

In [10]:
num_layers=2

In [11]:
W=torch.empty(num_layers,latent_dimensionality)
W=xavier_uniform_(W)
W

tensor([[ 0.5923,  0.2291],
        [-0.8256,  0.0123]])

In [12]:
drawer=qml.draw(variational_circuit)

In [13]:
# We draw the circuit. 
variational_circuit(latent_variables,W)
print(variational_circuit.draw())

 0: ──RX(0.182)──RZ(0.592)──╭C──RZ(-0.826)──╭C──┤ ⟨Z⟩ 
 1: ──RX(1.69)───RZ(0.229)──╰X──RZ(0.0123)──╰X──┤ ⟨Z⟩ 



In [14]:
variational_circuit(latent_variables,W)

tensor([ 0.4850, -0.2408], dtype=torch.float64)

# We have shown that the code for the variational circuit is working correctly. We can now proceed with the rest of the algorithm. 

In [15]:
# We define the Torch generator now.

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cuda:0 device


In [69]:
class QuantumGenerator(nn.Module):
    
    def __init__(self,variational_quantum_circuit,latent_dim,num_layers,upscaling_dimension,device):
        super(QuantumGenerator,self).__init__()
        
        self.device=device
        self.latent_dim=latent_dim
        # We initalize and store the quantum classifier's weights
        W=torch.Tensor(num_layers,latent_dim).uniform_(-np.pi,np.pi)
        self.quantum_weights=W
        
        # We store the quantum classifier
        self.vqc=variational_quantum_circuit
                
        # We define the upscaling layer, and we initialize it using the 
        # glorot uniform weight initialization
        self.upscaling_layer=nn.Linear(latent_dim,upscaling_dimension)
        xavier_uniform_(self.upscaling_layer.weight)
        
    def forward(self):
        # We define the latent variables, and pass them through a quantum generator.
        latent_variables=torch.Tensor(self.latent_dim).uniform_(-np.pi,np.pi).to(self.device)
        quantum_out=torch.Tensor(0,latent_dimensionality).to(self.device)
        exp_val=self.vqc(latent_variables,self.quantum_weights).float().unsqueeze(0).to(self.device)
        quantum_out=torch.cat((quantum_out,exp_val))
        generated_sample=self.upscaling_layer(quantum_out)
        return generated_sample

In [70]:
quantum_gen=QuantumGenerator(variational_circuit,latent_dimensionality,num_layers,4,device=device)
quantum_gen=quantum_gen.to(device)

In [71]:
quantum_gen()

tensor([[-1.7541, -1.7059, -0.4282,  0.4312]], device='cuda:0',
       grad_fn=<AddmmBackward>)

# We now define some toy loss functions to make sure that we know how to train the generator properly.